In [7]:
import pennylane as qml
from pennylane import numpy as np
import math

In [8]:
N = 1
n_wires = N+1
wires = range(n_wires)

q_wires = range(N, n_wires)
a_wires = range(0, N)

dev = qml.device("default.qubit", wires=wires, shots=50000)
U_alpha_mat = np.array([[1, 0], [0, 1]])
U_beta_mat = np.array([[0, 1], [1, 0]])


In [15]:
def U_alpha():
    return qml.QubitUnitary(U_alpha_mat, wires=q_wires)

def U_beta():
    return qml.QubitUnitary(U_beta_mat, wires=q_wires)

In [16]:
@qml.qnode(dev)
def hadamard_test(V_alpha, V_beta, W, component='real'):
    if component == 'imag':
        qml.RX(math.pi/2, wires=a_wires)
    
    qml.Hadamard(wires=a_wires)
    qml.QubitUnitary(W, wires=q_wires)

    def V_V_dagger():
        V_alpha()
        V_beta()

    V_V_dagger()    
    # qml.ControlledQubitUnitary(V_alpha.conjugate().T @ V_beta, control_wires=a_wires, wires=q_wires)
    qml.Hadamard(wires=a_wires)
    return qml.expval(qml.PauliZ(a_wires))

In [17]:
hadamard_test(U_alpha, U_beta, np.eye(2**N))

TypeError: __init__() got an unexpected keyword argument 'dagger'

In [122]:
def consistency_check_real_part(U_alpha, U_beta, W, component='real'):
    phi_alpha = U_alpha[0:-1, :]
    phi_beta = U_beta[0:-1, :]
    overlap = np.trace(phi_alpha.conjugate().T @ phi_beta)

    if component == 'real':
        return math.isclose(np.real(overlap), np.real(hadamard_test(U_alpha, U_beta, W, component='real')))
    elif component == 'imag':
        return math.isclose(np.imag(overlap), np.imag(hadamard_test(U_alpha, U_beta, W, component='imag')))


print(consistency_check_real_part(U_alpha, U_beta, np.eye(2**N), component='imag'))

True


In [123]:
unitaries = [U_alpha, U_beta]

approx_S = np.zeros((len(unitaries), len(unitaries)), dtype=complex)
for idx_1, u_1 in enumerate(unitaries):
    for idx_2, u_2 in enumerate(unitaries):
        approx_S[idx_1, idx_2] = hadamard_test(u_1, u_2, np.eye(2**N), component='real') + hadamard_test(u_1, u_2, np.eye(2**N), component='imag') * 1j

print(approx_S)
print(U_alpha.conjugate().T @ U_beta)


[[ 1.     -0.01148j -0.0024 -0.00392j]
 [-0.00436+0.00012j  1.     +0.00596j]]
[[0 1]
 [1 0]]
